In [ ]:
import pandas as pd
import numpy as np


In [ ]:
car=pd.read_csv('quikr_car.csv')

In [ ]:
car.shape

In [ ]:
car.head()

In [ ]:
car.info()

# Quality of data
1. Need to have consistent name 
2. Year has few non numeric values
3. Need to change data type of Year to Int
4. Company has few numeric values and dataype need to be changed to String
5. Price datatype is object need to change it to int
6. price has commas
7. kms in last need to remove, nan values, need to remove few records
8. fuel_type has nan values

In [ ]:
car=car[car['year'].str.isnumeric()]
car['year']=car['year'].astype(int)


## Price column

In [ ]:
car=car[car['Price']!='Ask For Price']

In [ ]:
car['Price'].unique()

In [ ]:
car['Price']=car['Price'].str.replace(',','').astype(int)

In [ ]:
car.info()

In [ ]:
car['kms_driven']=car['kms_driven'].str.split(' ').str.get(0).str.replace(',','')

In [ ]:
car=car[car['kms_driven'].str.isnumeric()]

In [ ]:
car.info()

In [ ]:
car['kms_driven']=car['kms_driven'].astype(int)

In [ ]:
car.info()

In [ ]:
car=car[~car['fuel_type'].isna()]

In [ ]:
car.info()

In [ ]:
car['name']=car['name'].str.split(' ').str.slice(0,3).str.join(' ')

In [ ]:
car=car.reset_index(drop=True)

In [ ]:
car

In [ ]:
car.describe()

In [ ]:
car.info()

In [ ]:
car[car['Price']>6e6]

In [ ]:
car=car[car['Price']<6e6].reset_index(drop=True)

In [ ]:
car

In [ ]:
car.to_csv('cleaned_car.csv')

Extracting Training data

In [ ]:
X=car[['name','company','year','kms_driven','fuel_type']]
y=car['Price']


In [ ]:
X

In [ ]:
y

## Training and Testing the data


In [ ]:
!pip3 install scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

In [ ]:
ohe=OneHotEncoder()
ohe.fit(X[['name','company','fuel_type']])

In [ ]:
column_trans = make_column_transformer(
    (OneHotEncoder(categories=ohe.categories_), ["name", "company", "fuel_type"]),
    remainder="passthrough",
)

In [ ]:
lr=LinearRegression()

In [ ]:
pipe=make_pipeline(column_trans,lr)

In [ ]:
pipe.fit(X_train,y_train)

In [ ]:
y_pred=pipe.predict(X_test)

In [ ]:
r2_score(y_test,y_pred)

finding the model with a random state


In [ ]:
scores=[]

for i in range(1000):
    X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=i)
    lr=LinearRegression()
    pipe=make_pipeline(column_trans,lr)
    pipe.fit(X_train,y_train)
    y_pred=pipe.predict(X_test)
    scores.append(r2_score(y_test,y_pred))

In [ ]:
np.argmax(scores)

In [ ]:
scores[np.argmax(scores)]

In [ ]:
pipe.predict(pd.DataFrame(columns=X_test.columns,data=np.array(['Hyundai Elite i20','Hyundai',2014,200,'Petrol']).reshape(1,5)))

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=np.argmax(scores))
lr=LinearRegression()
pipe=make_pipeline(column_trans,lr)
pipe.fit(X_train,y_train)
y_pred=pipe.predict(X_test)
r2_score(y_test,y_pred)